# Test du code Pytorch

In [3]:
import sys
sys.path.append('../src')
import shutil
import yaml
import py7zr
import os
import s3fs
from satellite_image import SatelliteImage
from labeler import RILLabeler
from labeler import BDTOPOLabeler
from labeled_satellite_image import SegmentationLabeledSatelliteImage
from utils import *
from filter import *
import matplotlib.pyplot as plt
import numpy as np
import rasterio.plot as rp
import re
from datetime import datetime
from tqdm import tqdm

In [4]:
update_storage_access()

## Téléchargement des données Pléiades

In [5]:
environment = get_environment()

bucket = environment["bucket"]
path_s3_pleiades_data = environment["sources"]["PLEIADES"]
path_s3_bdtopo_data = environment["sources"]["BDTOPO"][2022]["guyane"]
path_local_pleiades_data = environment["local-path"]["PLEIADES"]
path_local_bdtopo_data = environment["local-path"]["BDTOPO"][2022]["guyane"]

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

On télécharge les fichiers depuis s3 (à mettre dans le setup.sh?)

In [6]:
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data}",
        lpath=f"../{path_local_pleiades_data}",
        recursive=True)

In [5]:
fs.download(
        rpath=f"{bucket}/{path_s3_bdtopo_data}",
        lpath=f"../{path_local_bdtopo_data}",
        recursive=True)

## Découpage et filtrage des images

In [7]:
images_paths = [f"../{path_local_pleiades_data}/16bits/ORT_2022072050325085_U22N/" + p for p in os.listdir(f"../{path_local_pleiades_data}/16bits/ORT_2022072050325085_U22N/")]

In [8]:
date = datetime.strptime(re.search(r'ORT_(\d{8})', images_paths[0]).group(1), '%Y%m%d')

On importes les `SatelliteImage` dans une liste.

In [9]:
list_images = [
     SatelliteImage.from_raster(
        filename,
        dep = "973",
        date = date, 
        n_bands = 4
    ) for filename in tqdm(images_paths)]

On découpe les images initialement de dimension 2000 en plusieurs image de dimension `dim`. On ne garde ensuite que les images qui sont pertinentes à l'aide de la méthode `filter_images`. Une image contenant moins de 50% de pixels non noirs n'est pas retenue.

In [10]:
dim = 250

In [11]:
splitted_list_images = [im for sublist in tqdm(list_images) for im in sublist.split(250) if not is_too_black(im)]

In [12]:
len(splitted_list_images)

On peut représenter graphiquement nos images grâce à la méthode `plot` :

In [13]:
splitted_list_images[2793].plot([0, 1, 2]) # RGB (3 premières bandes)

On peut représenter l'image infrarouge : 

In [14]:
splitted_list_images[2793].plot([3, 0, 1]) # RGB (Infrarouge, rouge, vert)

Ou bien qu'une seule bande :

In [15]:
splitted_list_images[2793].plot([0]) # Une bande

## Labélisation avec le RIL

On définit un `labeler` incluant la taille des buffers que l'on souhaite ainsi que la forme de ceux-ci.

In [16]:
labeler_RIL = RILLabeler(date, dep="973", buffer_size=6, cap_style=3)

A l'aide de notre `labeler` on créer une `SegmentationLabeledSatelliteImage`.

In [91]:
list_labeled_images = []
for sat_im in tqdm(splitted_list_images[:30]):
    sat_im.normalize()
    list_labeled_images.append(
        SegmentationLabeledSatelliteImage(
            sat_im,
            labeler_RIL.create_segmentation_label(sat_im),
            "RIL",
            date
        )
    )

In [108]:
splitted_list_images[0].to_tensor().shape

In [109]:
splitted_list_images[0].to_tensor([0, 1, 2]).shape

### Dataset

In [110]:
%load_ext autoreload
%autoreload 2

In [111]:
from dataset import SatelliteDataset

In [114]:
from dataset import SatelliteDataModule

data_module = SatelliteDataModule(
    train_data=list_labeled_images[:20],
    test_data=list_labeled_images[20:],
    num_workers=0
)

In [115]:
train_dataloader = data_module.train_dataloader()

In [116]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

In [117]:
img = train_features[0].squeeze()

In [118]:
img.shape

In [ ]:
np.transpose(img[[0, 1, 2], :, :], (1, 2, 0))

In [122]:
plt.imshow(np.transpose(img[[0, 1, 2], :, :], (1, 2, 0)))
plt.show()

In [124]:
label = train_labels[0]
label.shape

In [125]:
plt.imshow(label, cmap="gray")
plt.show()

In [126]:
for label in train_labels:
    print(label.sum())

In [129]:
img = train_features[7].squeeze()

plt.imshow(np.transpose(img[[0, 1, 2], :, :], (1, 2, 0)))
plt.show()

In [130]:
label = train_labels[7]
plt.imshow(label, cmap="gray")
plt.show()

### Test avec BDTOPO

In [133]:
labeler_BDTOPO = BDTOPOLabeler(date, dep="973")

In [135]:
list_labeled_images = [
   SegmentationLabeledSatelliteImage(
      sat_im,
      labeler_BDTOPO.create_segmentation_label(sat_im),
      "BDTOPO",
      date
   ) for sat_im in tqdm(splitted_list_images[:30])]

In [136]:
data_module = SatelliteDataModule(
    train_data=list_labeled_images[:20],
    test_data=list_labeled_images[20:],
    num_workers=0
)

In [137]:
train_dataloader = data_module.train_dataloader()

In [138]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

In [139]:
img = train_features[0].squeeze()

In [140]:
img.shape

In [141]:
plt.imshow(np.transpose(img[[0, 1, 2], :, :], (1, 2, 0)))
plt.show()

In [142]:
label = train_labels[0]
label.shape

In [143]:
plt.imshow(label, cmap="gray")
plt.show()

### Add transforms

In [148]:
!pip install torchvision

In [149]:
import albumentations as album
from albumentations.pytorch.transforms import ToTensorV2

In [183]:
image_size = (250, 250)
# Some additional normalization is done here
transforms_augmentation = album.Compose(
    [
        album.Resize(300, 300, always_apply=True),
        album.RandomResizedCrop(
            *image_size, scale=(0.7, 1.0), ratio=(0.7, 1)
        ),
        album.HorizontalFlip(),
        album.VerticalFlip(),
        album.Normalize(),
        ToTensorV2(),
    ]
)

In [184]:
transforms_preprocessing = album.Compose(
    [
        album.Resize(*image_size, always_apply=True),
        album.Normalize(),
        ToTensorV2(),
    ]
)

In [185]:
data_module = SatelliteDataModule(
    train_data=list_labeled_images[:20],
    test_data=list_labeled_images[20:],
    transforms_augmentation=transforms_augmentation,
    transforms_preprocessing=transforms_preprocessing,
    num_workers=0,
    bands_indices=[0, 1, 2]
)

train_dataloader = data_module.train_dataloader()
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

In [186]:
test_dataloader = data_module.test_dataloader()
test_features, test_labels = next(iter(test_dataloader))